# Tratamento de dados do português

Esse notebook aborda o processamento sintático de textos em português brasileiro em Python 3.10 e abrange as etapas de importação, leitura, normalização, tokenização e, com maior atenção, lematização. O dataset analisado pode ser encontrado em [D&G UFF](https://deg.uff.br/corpus-dg/).

**Mariana Gonçalves da Costa** [Programa de Pós-Graduação em Informática/UFRJ]

Last updated: 29 December 2024

---

## Importação das bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import os
import nltk
import spacy
import json
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/Corpus_DeG")

print(os.getcwd())  # Imprime o diretório de trabalho atual

Mounted at /content/drive
/content/drive/MyDrive/Corpus_DeG


## Processamento do Texto

In [ ]:
def read_files(file):
  """ Lê o conteúdo de um arquivo .txt """

  with open(file, 'r', encoding='latin-1') as file:
    file = file.read()

    return file

In [ ]:
# Lê o conteúdo do dataset do D&G

juiz_de_fora = read_files('juiz_de_fora-1.txt')
natal_ = read_files('natal.txt')
niteroi_ = read_files('niteroi.txt')
rio_de_janeiro_a = read_files('rio_de_janeiro_a.txt')
rio_de_janeiro_b = read_files('rio_de_janeiro_b.txt')
rio_grande = read_files('rio_grande.txt')

In [ ]:
stopwordsdict = {
    "artigos": ["a", "o", "as", "os", "um", "uns", "uma", "umas"],
    "pronomes": ["eu", "tu", "ele", "ela", "nós", "vós", "eles", "elas", "você",
                "me", "te", "lhe", "o", "a", "se", "nos", "vos", "lhes", "os", "as", "se",
                "meu", "meus", "minha", "minhas", "teu", "tua", "teus", "tuas", "seu", "sua", "seus", "suas",
                "mim", "te", "ti", "nosso", "nossa", "nossos", "nossas", "dela", "dele", "deles", "delas",
                "que", "qual", "quais", "quem", "cujo", "cuja", "cujos", "cujas",
                "esse", "essa", "esses", "essas", "aquele", "isto", "isso", "aquilo",
                "aquela", "aqueles", "aquelas"],
    "preposicoes": ["com", "sem", "para", "pra", "pro", "pros", "de", "do", "da", "dos", "das", "em",
                    "no", "na", "nos", "nas", "num" "por", "que", "ao", "à"],
    "adverbios": ["assim", "aqui", "lá", "cá", "já", "logo", "muito", "muitos", "também",
                  "tanto", "quase", "sim", "não"],
    "conjuncoes": ["e", "mas", "entretanto", "porém", "pois", "porque", "ora", "ou", "logo",
                  "como", "seja", "todavia", "nem", "desde"],
    "verbos": ["ser", "sido", "sendo", "sou", "és", "é", "somos", "sois", "são",
              "era", "eras", "éramos", "éreis", "eram",
              "fui", "foste", "foi", "fomos", "fostes", "foram", "fora",
              "seja", "sejas", "sejamos", "sejais", "sejam", "fosse", "fôssemos", "fossem",
              "for", "formos", "forem",

              "estar", "estando", "está", "estado"
              "estou", "estás", "estamos", "estais", "estão",
              "estava", "estavas", "estávamos", "estáveis", "estavam",
              "estive", "estiveste", "esteve", "estivemos", "estivestes", "estiveram",
              "estivera", "estiveras", "estivéramos", "estivéreis", "estiveram",
              "estarei", "estarás", "estará", "estaremos", "estareis", "estarão",
              "estaria", "estarias", "estaríamos", "estaríeis", "estariam",
              "esteja", "estejas", "estejamos", "estejais", "estejam",
              "estivesse", "estivesses", "estivéssemos", "estivésseis", "estivessem",
              "estiver", "estiveres", "estivermos", "estiverdes", "estiverem"

              "ter", "tido", "tendo",
              "tenho", "tens", "tem", "temos", "tendes", "têm",
              "tinha", "tinhas", "tínhamos", "tínheis", "tinham",
              "tive", "tiveste", "teve", "tivemos", "tivestes", "tiveram",
              "tivera", "tiveras", "tivéramos", "tivéreis", "tiveram",
              "terei", "terás", "terá", "teremos", "tereis", "terão",
              "teria", "terias", "teríamos", "teríeis", "teriam",
              "tenha", "tenhas", "tenha", "tenhamos", "tenhais", "tenham",
              "tivesse", "tivesses", "tivéssemos", "tivésseis", "tivessem",
              "tiver", "tiveres", "tivermos", "tiverdes", "tiverem", "tende",

              "haver", "havido", "havendo", "hei", "hás", "há", "havemos", "haveis", "hão",
              "havia", "havias", "havíamos", "havíeis", "haviam",
              "houve", "houveste", "houvemos", "houvestes", "houveram",
              "houvera", "houveras", "houvéramos", "houvéreis",
              "haverei", "haverás", "haverá", "haveremos", "havereis", "haverão",
              "haveria", "haverias", "haveríamos", "haveríeis", "haveriam",
              "haja", "hajas", "hajamos", "hajais", "hajam",
              "houvesse", "houvesses", "houvéssemos", "houvésseis", "houvessem",
              "houver", "houveres", "houvermos", "houverdes", "houverem"],
    "marcador": ["ai", "aí", "né", "i", "ih", "ah", "oh", "eh"],
    "numeros" : ['1', '2', '3', '4', '5', '6', '7', '8', '9']
    }

# Sugestões de remoções: estado, estáveis, eras, tende.

# I organized the stopwords into a dictionary, grouping them by word classes.
# This structure allows you to select relevant stopwords for your data cleaning to align with the goals of text processing.

# Organizei as stopwords em um dicionário, agrupando-as pela classe de palavras.
# Essa estrutura permite selecionar stopwords relevantes para a sua limpeza de dados de acordo com os objetivos do processamento de texto.

In [ ]:
with open("stopwords.json", "w", encoding="utf-8") as json_file:
    json.dump(stopwordsdict, json_file, ensure_ascii=False, indent=4)

In [ ]:
def select_stopwords(stopwords, word_classes, remove_list = None):
  """Cria uma lista de stopwords selecionadas a partir de um dicionário.

  Argumentos:
    stopwordsdict (dict): Dicionário contendo todas as stopwords por classe de palavras

    word_classes (list): A lista de classes de palavras a serem incluídas nas stopwords finais.
    As classes podem ser:
    artigos, pronomes, preposicoes, adverbios, conjuncoes, verbos, marcadores discursivos, números

    remove_list (list, opcional): Uma lista com palavras a serem removidas das stopwords

  Retorna:
    stopwords (list): Uma lista final de stopwords.
  """

  stopwords = [word for word_class in word_classes for word in stopwordsdict.get(word_class, [])]
  # Use dict.get() para evitar KeyError caso a classe não se encontre no dicionário

  # Remove palavras selecionadas da list de stopwords caso o argumento remove_list seja preenchido
  # com uma lista de remoção

  if remove_list:
      stopwords = [word for word in stopwords if word not in remove_list]

  return stopwords

In [ ]:
stopwords = select_stopwords(stopwordsdict, ["artigos", "pronomes", "preposicoes", "adverbios", "conjuncoes", "marcador", "numeros"])

In [ ]:
# Pontuações e símbolos presentes nos arquivos de acordo com as diretrizes de
# transcrição do DeG

punctuation = ", . / \ [ ] ( ) : ; ? ! � … - \x94 \x93"

In [ ]:
def tokenization(text, punctuation):
    """Tokeniza as palavras em um texto a partir dos espaços em branco e dos separadores.

    Recebe:
        text (list): Lista de strings com o texto a ser tokenizado.
        punctuation (str): String contendo os separadores presentes no texto.

    Retorna:
        list: Uma lista de listas com os tokens (palavras) presentes em cada bloco de texto.
    """

    # Cria uma lista com os separadores, que estavam cada um em uma linha
    caracteres = punctuation.split()

    # Elimina elementos vazios da lista de caracteres
    caracteres = list(filter(None, caracteres))

    # Cria um dicionário com as substituições que precisarão ser feitas na string
    trocar = {}

    for c in caracteres:
      trocar.update(text.maketrans(c, " "))

    # Realiza as substituições (por espaço) e quebra a string em uma lista de tokens
    tokens = text.translate(trocar).split()

    return tokens

In [ ]:
def normalizacao(palavras):
  """ Normaliza uma lista de palavras.

  Recebe: uma lista de palavras.

  Retorna: uma lista de palavras normalizadas.
  """

  return [palavra.lower() for palavra in palavras]

In [ ]:
def preprocessing(text, punctuation):
  """ Executa o pré-processamento de um texto.

  Recebe: uma lista com o texto e uma lista de caracteres separadores.

  Retorna: uma lista de palavras pré-processadas referentes ao texto passado.
  """

  tokens = tokenization(text, punctuation)

  tokens_normais = normalizacao(tokens)

  return tokens_normais

In [ ]:
import copy

# Cria uma deepcopy dos originais
juizdefora = copy.deepcopy(juiz_de_fora)
natal = copy.deepcopy(natal_)
niteroi = copy.deepcopy(niteroi_)
riodejaneiroa = copy.deepcopy(rio_de_janeiro_a)
riodejaneirob = copy.deepcopy(rio_de_janeiro_b)
riogrande = copy.deepcopy(rio_grande)

# Cria uma versão unificada do dataset
baseconjugada = juizdefora + natal_ + niteroi_ + riodejaneiroa + riodejaneirob + riogrande

# Imprime o tamanho de cada set do dataset após o pré-processamento
print("Pré-processamento da Base Conjugada:")
baseconjugada = preprocessing(baseconjugada, punctuation)
print(len(baseconjugada))

print("Pré-processamento da Base Juiz de Fora:")
juizdefora = preprocessing(juiz_de_fora, punctuation)
print(len(juizdefora))

print("Pré-processamento da Base Natal:")
natal = preprocessing(natal_, punctuation)
print(len(natal))

print("Pré-processamento da Base Niterói:")
niteroi = preprocessing(niteroi_, punctuation)
print(len(niteroi))

print("Pré-processamento da Base Rio de Janeiro A:")
riodejaneiroa = preprocessing(rio_de_janeiro_a, punctuation)
print(len(riodejaneiroa))

print("Pré-processamento da Base Rio de Janeiro B:")
riodejaneirob = preprocessing(rio_de_janeiro_b, punctuation)
print(len(riodejaneirob))

print("Pré-processamento da Base Rio Grande:")
riogrande = preprocessing(rio_grande, punctuation)
print(len(riogrande))

Pré-processamento da Base Conjugada:
447290
Pré-processamento da Base Juiz de Fora:
53653
Pré-processamento da Base Natal:
155941
Pré-processamento da Base Niterói:
38028
Pré-processamento da Base Rio de Janeiro A:
76155
Pré-processamento da Base Rio de Janeiro B:
91351
Pré-processamento da Base Rio Grande:
32162


In [ ]:
with open('riogrande.txt', 'w') as file:
    for item in riogrande:
        file.write(item + '\n')

In [ ]:
with open('baseconjugada.txt', 'w') as file:
    for item in baseconjugada:
        file.write(item + '\n')

In [ ]:
def elimina_stopwords(palavras, stopwords):
    """ Remove stopwords de uma lista de palavras.

    Recebe: uma lista de palavras e uma lista de stopwords.

    Retorna: uma lista de palavras sem as stopwords.
    """
    stopwords_set = set(stopwords)
    return [palavra for palavra in palavras if palavra not in stopwords_set]

In [ ]:
# Imprime o tamanho de cada set do dataset após o a remoção de stopwords
print("Remoção das stopwords da Base Conjugada:")
baseconjugada_sem_stopwords = elimina_stopwords(baseconjugada, stopwords)
print(len(baseconjugada_sem_stopwords))

print("Remoção das stopwords da da Base Juiz de Fora:")
juizdefora_sem_stopwords = elimina_stopwords(juizdefora, stopwords)
print(len(juizdefora_sem_stopwords))

print("Remoção das stopwords da da Base Natal:")
natal_sem_stopwords = elimina_stopwords(natal, stopwords)
print(len(natal_sem_stopwords))

print("Remoção das stopwords da Base Niterói:")
niteroi_sem_stopwords = elimina_stopwords(niteroi, stopwords)
print(len(niteroi_sem_stopwords))

print("Remoção das stopwords da Base Rio de Janeiro A:")
riodejaneiroa_sem_stopwords = elimina_stopwords(riodejaneiroa, stopwords)
print(len(riodejaneiroa_sem_stopwords))

print("Remoção das stopwords da Base Rio de Janeiro B:")
riodejaneirob_sem_stopwords = elimina_stopwords(riodejaneirob, stopwords)
print(len(riodejaneirob_sem_stopwords))

print("Remoção das stopwords da Base Rio Grande:")
riogrande_sem_stopwords = elimina_stopwords(riogrande, stopwords)
print(len(riogrande_sem_stopwords))

Remoção das stopwords da Base Conjugada:
249316
Remoção das stopwords da da Base Juiz de Fora:
28969
Remoção das stopwords da da Base Natal:
87808
Remoção das stopwords da Base Niterói:
21156
Remoção das stopwords da Base Rio de Janeiro A:
43758
Remoção das stopwords da Base Rio de Janeiro B:
49838
Remoção das stopwords da Base Rio Grande:
17787


In [ ]:
with open('riogrande_sem_stopwords.txt', 'w') as file:
    for item in riogrande_sem_stopwords:
        file.write(item + '\n')

---

## Abordagens de lematização


1.   **Lematização por aprendizado de máquina**: spaCy modelo pt_core_news_sm
2.   **Lematização por depêndencias universais**: Simplemma UD-PT-GSD
3.   **Lematização por arquivos lexicográficos**: PortiLexicon-UD (Projeto POeTiSA)


## Importação do modelo do português do spaCy

In [ ]:
!python -m spacy download pt_core_news_sm --quiet

nlp = spacy.load('pt_core_news_sm')

nlp.max_length = 3000000

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 80.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Lematização por modelo e UD: spaCy

In [ ]:
def lemmatizationspacy(tokens):
  """ Lematiza um texto usando a biblioteca spaCy.

  Recebe: uma lista de tokens (str).

  Retorna: uma lista com os tokens lematizados.
  """

  doc = nlp(' '.join(tokens))
  lemmas = [token.lemma_ for token in doc]  # Lista com os lemas dos tokens

  return lemmas

In [ ]:
lema_spacy1 = lemmatizationspacy(riogrande)

In [ ]:
lema_spacy2 = lemmatizationspacy(riogrande_sem_stopwords)

In [ ]:
lema_spacy3 = lemmatizationspacy(baseconjugada)

In [ ]:
with open("lema_spacy1.txt", "w") as file:
    for item in lema_spacy1:
        file.write(item + '\n')

In [ ]:
with open("lema_spacy2.txt", "w") as file:
    for item in lema_spacy2:
        file.write(item + '\n')

In [ ]:
with open("lema_spacy3.txt", "w") as file:
    for item in lema_spacy3:
        file.write(item + '\n')

### Lematização com [Simplemma](https://github.com/adbar/simplemma)

In [ ]:
!pip install git+https://github.com/adbar/simplemma --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import simplemma

def simplemmatization(texto):
  """ Lematiza um texto através da função simplemmatization do simplemma.

  Recebe: um texto (string).

  Retorna: um dataframe com o texto original e o texto lematizado.
  """

  lemmas = []
  for token in simplemma.lemmatize(texto, lang = 'pt'):
    lemmas.append(token)
  return ''.join(lemmas)

In [ ]:
# Criar DataFrame
simlematizacao = pd.DataFrame({'Texto': riogrande})

# Aplicar lematização aos dados
simlematizacao['Texto Lematizado'] = simlematizacao['Texto'].apply(simplemmatization)

In [ ]:
# Criar DataFrame
simlematizacao2 = pd.DataFrame({'Texto': riogrande_sem_stopwords})

# Aplicar lematização aos dados
simlematizacao2['Texto Lematizado'] = simlematizacao2['Texto'].apply(simplemmatization)

In [ ]:
# Criar DataFrame
simlematizacao3 = pd.DataFrame({'Texto': baseconjugada})

# Aplicar lematização aos dados
simlematizacao3['Texto Lematizado'] = simlematizacao2['Texto'].apply(simplemmatization)

In [ ]:
lema_simp1 = simlematizacao['Texto Lematizado'].values.tolist()

In [ ]:
lema_simp2 = simlematizacao2['Texto Lematizado'].values.tolist()

In [ ]:
lema_simp3 = simlematizacao3['Texto Lematizado'].values.tolist()

In [ ]:
with open("lema_simp1.txt", "w") as file:
    for item in lema_simp1:
        file.write(item + '\n')

In [ ]:
with open("lema_simp2.txt", "w") as file:
    for item in lema_simp2:
        file.write(item + '\n')

In [ ]:
with open("lema_simp3.txt", "w") as file:
    for item in lema_simp3:
       if not pd.isna(item):
            file.write(str(item) + '\n') # Converte item em string se não for NaN

### Lematização com PortiLexicon-UD

In [ ]:
portilexicon = pd.read_csv('portilexicon-ud - portilexicon-ud.csv')

In [ ]:
def lematizacao_portilexicon(texto, lexicon = portilexicon):
    """Lematiza um texto através do documento lexicográfico PortiLexicon.

    Recebe:
        texto (list): Lista de tokens (strings) do texto.
        portilexicon (pd.DataFrame): DataFrame contendo as colunas 'a' (token) e 'a.1' (lemma).

    Retorna:
        list: Lista de tokens lematizados.
    """
    # Verificar se o portilexicon foi fornecido corretamente
    if portilexicon is None:
        raise ValueError("O parâmetro 'portilexicon' não foi fornecido.")

    # Criar um dicionário para busca rápida dos lemas
    lemma_dict = dict(zip(portilexicon['a'], portilexicon['a.1']))

    # "Flatten" do texto: se for uma lista de listas, transformar em uma lista plana
    if isinstance(texto[0], list):
        texto = [item for sublist in texto for item in sublist]

    # Substituir tokens pelo lema correspondente ou manter o token original
    return [lemma_dict.get(token, token) for token in texto]

In [ ]:
lema_port1 = lematizacao_portilexicon(riogrande)

In [ ]:
lema_port2 = lematizacao_portilexicon(riogrande_sem_stopwords)

In [ ]:
lema_port3 = lematizacao_portilexicon(baseconjugada)

In [ ]:
with open("lema_port1.txt", "w") as file:
    for item in lema_port1:
        file.write(item + '\n')

In [ ]:
with open("lema_port2.txt", "w") as file:
    for item in lema_port2:
        file.write(item + '\n')

In [ ]:
with open("lema_port3.txt", "w") as file:
    for item in lema_port3:
        file.write(item + '\n')

## Validação

In [ ]:
testeriogrande = [riogrande[:1000]]

In [ ]:
lematizacaomodelo = ['informante', 'de', 'ensino', 'superior', 'informante', '1', 'joão', 'carlos', 'sexo', 'masculino', 'idade', '28', 'ano', 'data', 'de', 'coleta', 'oral', '20', '11', '93', 'escrita', '21', '11', '93', 'e',
'22', '11', '93', 'parte', 'oral', 'narrativa', 'de', 'experiência', 'pessoal', 'e', 'joão', 'ter', 'algum', 'história', 'que', 'ter', 'acontecer', 'contigo', 'que', 'tu', 'querer', 'eu', 'contar', 'i',
'ter', 'um', 'recente', 'eh', 'pegar', 'o', 'lancha', 'aí', 'ir', 'eu', 'e', 'dois', 'amigo', 'meu', 'que', 'trabalhar', 'comigo', 'lá', 'o', 'edson', 'e', 'o', 'miguel', 'aí',
'entrar', 'em', 'lancha', 'né', 'ter', 'não', 'saber', 'se', 'você', 'saber', 'como', 'ser', 'que', 'o', 'lancha', 'ter', 'quatro', 'um', 'fileira', 'de','quatro', 'cadeira','assim',
'de', 'madeira', 'com', 'encosto', 'e', 'o', 'cadeira', 'ser', 'normal', 'aí', 'até', 'ir', 'entrar', 'conversar', 'aí','pedir', 'pra', 'dar', 'lugar', 'pra', 'outro', 'sentar', 'também', 'sentar', 'em', 'canto', 'o', 'outro', 'dois', 'sentar',
'perto','de', 'eu', 'só', 'que', 'eu', 'não', 'ver', 'que', 'estar', 'sem', 'encosto', 'o', 'cadeira', 'eu', 'sentar', 'o','cadeira', 'ser', 'de', 'ser', 'imóvel', 'vir', 'pra', 'trás', 'cair', 'deitado', 'em', 'colo', 'de',
'negrão', 'atrás', 'de', 'eu', 'riso', 'aquilo', 'ali', 'ficar', 'o', 'vale', 'de', 'ano', 'lá', 'em', 'em', 'narrativa', 'recontado', 'e', 'estar', 'joão',  'e', 'ter', 'algum', 'história', 'que', 'alguém',
'ter', 'te', 'contar', 'e', 'que', 'tu', 'não', 'ter', 'participado', 'e', 'que', 'tu', 'saber', 'através', 'de', 'alguém', 'i', 'eh', 'eu', 'não', 'saber', 'se', 'chegar', 'a', 'ser', 'o', 'que', 'tu', 'estar', 'querer',
'mas', 'isso', 'acontecer', 'sexta', 'feira', 'também', 'ser', 'mais', 'pra',  'mais', 'um', 'vale', 'datar', 'o', 'cara', 'chegar', 'estar', 'eh', 'almoçar', 'o', 'mãe', 'dele', 'ligar', 'pra', 'ele', 'assim', 'está',
'com', 'um', 'pé', 'de', 'cada','sapato','aí','o','o','colega','chegar','e','disse','ué','pé','de','cada','sapato','você','achar','que','eu','ser','louco','por','um','pé','de','sapato','um','pé','de',
'cada','sapato','olhar','pra','baixo','um','sapato','com','um','lista','e','outro','sem','lista','riso','de','e','ir','trabalhar','com','um','pé','de','cada','sapato','pegar','em','pé','dele','lá','descrição',
'de','local','e','tá','joão','eh','ter','algum','lug','um','lugar','ou','algum','coisa','que','tu','querer','eu','descrever','i','eu','poder','você','descrever','o','agência','lá','em','em','norte','onde',
'eu','trabalhar','eh','logo','em','seguida','qur','tu','descer','de','lancha','ter','o','o','rua','de','frente','ali','álvaro','costa','o','agência','ser','bem','em','frente','ser','dois','piso','né','o',
'setor','de','baixo','ser','onde','funcionar','o','o','caixa','de','gerência','e','em','cima','o','pessoal','mais','o','pessoal','de','de','aplicação','ser','um','um','agência','pequeno','mas','eh','bem',
'bonito','até','relato','de','procedimento','e','tá','e','ter','algum','coisa','que','tu','querer','eu','ensinar','a','fazer','explicar','explicar','como','faz','algum','coisa','pode','ser','algum','coisa',
'relacionado','com','o','você','serviço','algum','i','ser','ser','difícil','ir','dizer','você','ensinar','a','trabalhar','em','caixa','o','o','rapaz','que','fazer','compensação','por','exemplo','quando',
'você','chegar','com','um','depósito','com','cheque','de','outro','banco','tu','tu','tem','que','carimbar','o','cheque','né','aí','em','final','de','dia','tu','tu','somar','ele','banco','por','banco','que',
'ficar','só','pra','pra','compensação','eh','ser','coisa','entender','que','eu','poder','até','você','explicar','mas','não','relato','de','opinião','e','aí','tu','querer','dar','um','opinião','sobre',
'algum','coisa','algum','assunto','i','eh','como','eu','trabalhar','em','banco','agora','que','falar','tudo','agora','sobre','o','sobre','banco','ir','você','dar','um','motivo','um','opinião','sobre',
'não','sei','aplicação','eu','achar','que','pelo','menos','ser','de','se','esperar','como','o','taxa','estar','muito','alta','em','aplicação','dever','ter','um','mudança','em','plano','econômico','de',
'de','país','pro','final','de','ano','até','estar','falar','lá','em','algum','pacote','algum','coisa','porque','a','taxa','estar','muito','al','muito','alta','muito','acima','de','inflação','o','inflação',
'que','ele','divulgar','né','então','por','a','taxa','estar','tão','alta','certamente','dever','ter','algum','mudança','em','em','plano','econômico','e','tá','i','bom','dia','e','obrigado','parte','escrito',
'narrativa','de','experiência','pessoal','ir','descrever','um','acontecimento','engraçado','que','se','passar','comigo','em','lancha','para','são','josé','do','norte','quando','ia','para','o','serviço',
'a','lancha','noivo','de','mar','ter','fileira','com','4','banco','de','madeira','e','o','assento','ser','móvel','entrar','em','lancha','juntamente','com','dois','colega','de','serviço','e','sentei','em',
'cadeira','de','canto','porém','não','ver','que','o','cadeira','estar','sem','encosto','cair','com','tudo','pra','trás','em','colo','de','um','senhor','que','estar','no','banco','de','trás','ser','o',
'vale','de','ano','narrativa','recontado','outro','acontecimento','engraçado','acontecer','outro','dia','com','um','colega','em','serviço','o','mãe','dele','ligar','pra','ele','perguntar','se','ele','estar',
'com','um','pé','de','cada','sapato','ele','ainda','ficar','bravo','dizer','que','não','estar','louco','e','olhar','para','o','pé', 'estar','mesmo','com','um','pé','de','cada','imaginar','o','situação',
'descrição','de','local','ir','descrever','o','agência','de','caixa','econômico','federal','em','são','josé','de','norte','ser','logo','em','rua','em','frente','o','hidroviário','um','agência','de','dois',
'andar','onde','em','parte','inferior','ficar','localizado','o','gerência','o','abertura','de','conta','e','o','caixa','em','andar','superior','ficar','o','cozinha','e','o','banheiro','o','almoxarifado',
'e','o','setor','de','contabilidade','e','aplicação','ser','um','agência','pequeno','porém','bem','agradável','relato','de','procedimento','ir','dar','um','pequeno','procedimento','sobre','o','tratamento',
'dado','a','cheque','de','outro','banco','recebido','pelo','caixa','em','depósito','o','caixa','recebe','o','depósito','em','cheque','carimbar','o','cheque','com','o','carimbo','de','compensação','em',
'verso','e','com','o','de','cruzamento','em','frente','verificar','o','prazo','de','comp','bloqueio','e','autenticar','o','depósito','em','final','de','expediente','fazer','o','soma','de','cheque','verificar',
'se','coincidir','com','o','total','de','depósito','recebido','e','encaminhar','para','o', 'retaguarda','relato', 'de', 'opinião', 'ir', 'dar','um', 'opinião','sobre','o','situação', 'econômico', 'que',
'provavelmente', 'dever', 'ter', 'mudança', 'em', 'final', 'desse', 'ano', 'visto', 'que', 'o', 'taxa', 'de', 'juro','em', 'aplicação', 'em', 'cdb', 'rdb', 'poupança', 'e','outro','estar','muito','elevado',
'em','relação','o', 'inflação', 'divulgado', 'achar', 'que', 'certamente', 'este', 'equilíbrio', 'provocar', 'mudança', 'e', 'quem', 'saber', 'um', 'novo','pacote', 'econômico', 'informante', '2',
'lisandra','sexo','feminino','idade', '23','ano','data','de','coleta','oral','28','09','93','escrita','06','11','93','parte','escrito','narrativa','de','experiência','pessoal','e','lisandra', 'um',
'experiência', 'um', 'narrativa', 'teu', 'de', 'experiência']

In [ ]:
def contaacuracia(lematizacaomodelo, lematizacao):

  corretos = 0

  for i in range(len(lematizacaomodelo)):
    if lematizacaomodelo[i] == lematizacao[i].lower():
      corretos += 1

  acuracia = corretos / len(lematizacaomodelo)

  return acuracia

In [ ]:
contaacuracia(lematizacaomodelo, lema_spacy1)

0.869

In [ ]:
contaacuracia(lematizacaomodelo, lema_simp1)

0.823

In [ ]:
contaacuracia(lematizacaomodelo, lema_port1)

0.8

In [ ]:
lematizacaomodelo2 = elimina_stopwords(lematizacaomodelo, stopwords)
len(lematizacaomodelo2)

590

In [ ]:
verificacao = pd.DataFrame({'Texto': lematizacaomodelo2, 'Texto Lematizado': lema_port2[:590]})

In [ ]:
verificacao.to_csv("verificacao.csv", index=False)

In [ ]:
contaacuracia(lematizacaomodelo2, lema_spacy2)

0.8661016949152542

In [ ]:
contaacuracia(lematizacaomodelo2, lema_simp2)

0.8627118644067797

In [ ]:
contaacuracia(lematizacaomodelo2, lema_port2)

0.8067796610169492

---

In [ ]:
from itertools import chain

def frequencia_ngramas(base, n = 2):
  """ Identifica os ngramas de um dataset e os ranqueia pela frequência.

  Recebe: base (list): Uma lista de palavras ou tokens (strings).
        n (int): O número de tokens para o n-grama. Valor padrão é 2.

  Retorna: list: Uma lista de tuplas (n-grama, frequência) ordenada por frequência em ordem decrescente.
  """

  # Validações de entrada
  if not isinstance(base, list) or not all(isinstance(item, (str, list)) for item in base):
      raise ValueError("A base deve ser uma lista de strings ou uma lista de listas de strings.")
  if not isinstance(n, int) or n < 1:
      raise ValueError("O valor de 'n' deve ser um inteiro positivo.")

  # Se for uma lista de listas, achata para uma única lista
  if any(isinstance(item, list) for item in base):
      base = list(chain.from_iterable(base))

  counts = {}

  for ngram in nltk.ngrams(base, n):  # Cria os n-gramas
      counts[ngram] = counts.get(ngram, 0) + 1

  ngramsranking = sorted(counts.items(), key=lambda x: x[1], reverse=True)

  return ngram, ngramsranking

In [ ]:
frequencia_ngramas(riogrande)

(('estas', 'coisas'),
 [(('que', 'eu'), 132),
  (('a', 'gente'), 116),
  (('que', 'tu'), 109),
  (('e', 'o'), 84),
  (('que', 'é'), 84),
  (('e', 'eu'), 81),
  (('relato', 'de'), 75),
  (('e', 'a'), 74),
  (('e', 'e'), 74),
  (('a', 'minha'), 66),
  (('o', 'que'), 63),
  (('é', 'um'), 63),
  (('alguma', 'coisa'), 62),
  (('acho', 'que'), 57),
  (('o', 'meu'), 55),
  (('tem', 'um'), 53),
  (('com', 'a'), 52),
  (('que', 'a'), 51),
  (('é', 'uma'), 51),
  (('que', 'que'), 49),
  (('coisa', 'que'), 47),
  (('né', 'e'), 47),
  (('tem', 'que'), 46),
  (('eu', 'acho'), 46),
  (('eu', 'não'), 44),
  (('experiência', 'pessoal'), 43),
  (('tem', 'uma'), 42),
  (('que', 'não'), 42),
  (('de', 'experiência'), 40),
  (('e', 'não'), 40),
  (('é', 'o'), 40),
  (('narrativa', 'de'), 39),
  (('é', 'que'), 39),
  (('em', 'cima'), 39),
  (('de', 'opinião'), 38),
  (('narrativa', 'recontada'), 37),
  (('de', 'local'), 37),
  (('minha', 'mãe'), 37),
  (('descrição', 'de'), 36),
  (('de', 'procedimento'), 

In [ ]:
frequencia_ngramas(riogrande_sem_stopwords)

(('estas', 'coisas'),
 [(('alguma', 'coisa'), 62),
  (('experiência', 'pessoal'), 43),
  (('narrativa', 'experiência'), 40),
  (('relato', 'opinião'), 39),
  (('narrativa', 'recontada'), 37),
  (('relato', 'procedimento'), 36),
  (('descrição', 'local'), 35),
  (('é', 'é'), 27),
  (('11', '93'), 26),
  (('todo', 'mundo'), 25),
  (('anos', 'data'), 19),
  (('data', 'coleta'), 19),
  (('coleta', 'oral'), 19),
  (('93', 'parte'), 19),
  (('parte', 'escrita'), 19),
  (('escrita', 'narrativa'), 19),
  (('por', 'causa'), 19),
  (('93', 'escrita'), 18),
  (('parte', 'oral'), 18),
  (('oral', 'narrativa'), 18),
  (('vamos', 'dizer'), 17),
  (('é', 'lugar'), 17),
  (('é', 'bem'), 16),
  (('tem', 'tem'), 16),
  (('gente', 'tem'), 15),
  (('tem', 'alguma'), 14),
  (('rio', 'grande'), 14),
  (('mais', 'menos'), 14),
  (('guarda', 'roupa'), 14),
  (('porto', 'alegre'), 13),
  (('acho', 'é'), 13),
  (('opinião', 'sobre'), 12),
  (('09', '93'), 12),
  (('é', 'coisa'), 12),
  (('é', 'bom'), 12),
  (('